## Modifying Data in the Data Frame

In [50]:
import pandas as pd
import numpy as np

In [51]:
coffee = pd.read_csv('./data/coffee.csv')
coffee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Day          14 non-null     object
 1   Coffee Type  14 non-null     object
 2   Units Sold   14 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 464.0+ bytes


In [52]:
# Adding a column called price, all records are 4.99
coffee['price'] = 4.99

# New column, but using numpy, assigns 3.99 is it's Espresso and 5.99 if otherwise
coffee['new_price'] = np.where(coffee['Coffee Type'] == "Espresso", 3.99, 5.99)

# Drop that old price column
# DOESN'T actually modify the data frame, just the output
# Adding inplace=True, deletes it for real
coffee.drop(columns=['price'], inplace=True)

# This also does the same as dropping, you just reassign the data from with the cols you want
coffee = coffee[['Coffee Type', 'Units Sold', 'new_price']]

# Adds a new column using some of the old ones to make values
coffee['revenue'] = coffee['Units Sold'] * coffee['new_price']

# Rename a column, inplace modifies it for real
coffee.rename(columns={'new_price':'price'}, inplace=True)
# This is the same: coffee = coffee.rename(columns={'new_price':'price'})

In [53]:
coffee.head().sort_values('revenue', ascending=False)

,Coffee Type,Units Sold,price,revenue
4,Espresso,35,3.99,139.65
3,Latte,20,5.99,119.80
2,Espresso,30,3.99,119.70
0,Espresso,25,3.99,99.75
1,Latte,15,5.99,89.85


In [54]:
# New data set

bios = pd.read_excel('./data/olympics-data.xlsx')
bios.head()

,athlete_id,name,born_date,born_city,born_region,born_country,NOC,height_cm,weight_kg,died_date
0,1,Jean-François Blanchy,1886-12-12,Bordeaux,Gironde,FRA,France,NaN,NaN,1960-10-02
1,2,Arnaud Boetsch,1969-04-01,Meulan,Yvelines,FRA,France,183.0,76.0,NaN
2,3,Jean Borotra,1898-08-13,Biarritz,Pyrénées-Atlantiques,FRA,France,183.0,76.0,1994-07-17
3,4,Jacques Brugnon,1895-05-11,Paris VIIIe,Paris,FRA,France,168.0,64.0,1978-03-20
4,5,Albert Canet,1878-04-17,Wandsworth,England,GBR,France,NaN,NaN,1930-07-25


In [55]:
# Add a new column with the first name of the athlete, pulled from their whole name
bios['first_name'] = bios['name'].str.split(' ').str[0]
bios['last_name'] = bios['name'].str.split(' ').str[1]

# Add a new column of born_date but in datetime format, makes it modifiable
bios['born_datetime'] = pd.to_datetime(bios['born_date'])
bios['born_year'] = bios['born_datetime'].dt.year

# Then find anyone with the first name Jack
bios.query('first_name == "Jack"')

# First five records in bios, only those columns, decending by year
# Also reassign the data frame to be this so we can export it
newbios = bios[['first_name', 'last_name', 'born_year']].sort_values('born_year', ascending=False).head()

# Now we save that above to a new csv, don't include the index values
newbios.to_csv('./data/bios_new.csv', index=False)

In [58]:
# Custom columns

# This will make a new column, then apply a cetogry based on the height of someone
# x represents the hight_cm in the lambda function
bios['height_category'] = bios['height_cm'].apply(lambda x: 'Short' if x < 165
                                                  else ('Average' if x < 185 else 'Tall'))

# Using a function to categorize
def categroize_athlete(row):
    if row['height_cm'] < 175 and row['weight_kg'] < 70:
        return 'Lightweight'
    elif row['height_cm'] < 185 or row['weight_kg'] <= 80:
        return 'Middleweight'
    else:
        return 'Heavyweight'
    
# Using the function in the .apply function (1 is rows, 0 is columns)
bios['Category'] = bios.apply(categroize_athlete, axis=1)

bios.head()

,athlete_id,name,born_date,born_city,born_region,born_country,NOC,height_cm,weight_kg,died_date,first_name,last_name,born_datetime,born_year,height_category,Category
0,1,Jean-François Blanchy,1886-12-12,Bordeaux,Gironde,FRA,France,NaN,NaN,1960-10-02,Jean-François,Blanchy,1886-12-12,1886.0,Tall,Heavyweight
1,2,Arnaud Boetsch,1969-04-01,Meulan,Yvelines,FRA,France,183.0,76.0,NaN,Arnaud,Boetsch,1969-04-01,1969.0,Average,Middleweight
2,3,Jean Borotra,1898-08-13,Biarritz,Pyrénées-Atlantiques,FRA,France,183.0,76.0,1994-07-17,Jean,Borotra,1898-08-13,1898.0,Average,Middleweight
3,4,Jacques Brugnon,1895-05-11,Paris VIIIe,Paris,FRA,France,168.0,64.0,1978-03-20,Jacques,Brugnon,1895-05-11,1895.0,Average,Lightweight
4,5,Albert Canet,1878-04-17,Wandsworth,England,GBR,France,NaN,NaN,1930-07-25,Albert,Canet,1878-04-17,1878.0,Tall,Heavyweight
